<img style="float: left;padding-right: 10px" width ="40px" src="https://raw.githubusercontent.com/bartczernicki/DecisionIntelligence.GenAI.Workshop/main/Images/SemanticKernelLogo.png">

## Semantic Kernel - Premortem (What-If) Decision Analysis  

> 📜 **_"The pre-mortem method has a number of benefits besides identifying trouble spots in a plan – reducing overconfidence and promoting discoveries. The method can strengthen members’ mental models… And it can help create a culture of candor and trust."_**
>
> -- <cite>Gary Klein (Psychologist, creator of the Pre-mortem method)</cite>  

Decision Intelligence applied in this module:  
* Using Premortem Decision Analysis to improve decision quality & outcomes  
* Using a Premortem decision output for improving of Gathering Intelligence 
* Attain different perspectives using mutliple decision frames  
* Apply a Decision Framework to Premortem Decision Analysis 


### What is a Premortem (What-If Analysis)  

You have probably heard of a **postmortem (post-mortem)**, which is a look back at an actual failure (or success) to understand why it happened and learn from it. In the context of decision making, after the decision outcome is clear you can analyze why the result occured. For example, a hedge fund may host a postmortem meeting after analyzing an unsuccesful move on a tech stock that didn't materialize the return on investment as was planned. 

In a **premortem (pre-mortem)** the outcome is "imagined" and then it is figured out why that potential "imagined" outcome happened. In a premortem, a planning exercise is held where a team imagines that a project or decision has already succeeded or failed. The team then works backward, brainstorming what might have caused things to go wrong. This approach allows everyone to spot potential risks and address them before starting the project. Therefore, it’s a bit like predicting the future and using that prediction to make the forecast stronger. 

### Example Decision Scenario - Purchasing a Car   

Imagine you're about to buy a new car. Let's play a game of "What-If" that the purchase decision results in either two outcomes a: **disaster** or a **successful** outcome. 

#### Premortem Outcome - Disaster  

<img style="display: block; margin: auto;" width ="700px" src="https://raw.githubusercontent.com/bartczernicki/DecisionIntelligence.GenAI.Workshop/main/Images/Scenarios/Scenario-DecisionPremortem.png">

In a **premortem (pre-mortem)**, you pretend the car purchase turned out to be a disaster. Now, you ask yourself:  
**What went wrong with the decision to buy a new car?**

Here are some potential reasons the car purchase had a poor outcome:  

* **Financial issues:** You realize later you can’t comfortably afford the monthly payments or maintenance costs.  
* **Unexpected repairs:** The car has hidden mechanical problems, leading to costly repairs.  
* **Insurance costs:** Insurance is much higher than expected, impacting your budget.
* **Poor fuel efficiency:** The car uses more gas than you thought, making it expensive to drive.
Resale value drop: The car’s value plummets quickly, leaving you with less if you want to sell or trade it.  

#### Premortem Outcome - Success

In a premortem for a successful car purchase, imagine you bought the car, and it turned out to be a succssful decision. Now, you ask yourself:  
**What went right with the decision to buy a new car?**  

Here are some potential reasons the car purchase had a good outcome: 

* **Affordable and realistic budget:** You set a budget that allowed for comfortable monthly payments and covered insurance, fuel, and maintenance without straining your finances.  
* **Thorough inspection:** You got a detailed inspection before buying, which confirmed the car was in excellent condition and minimized the risk of future repairs.
* **Reliable model choice:** You researched models with a reputation for reliability, so you experience fewer repairs and smooth performance.
* **Fuel efficiency matches needs:** You chose a car that meets your fuel efficiency needs, saving money on gas and fitting well with your daily driving habits.   
* **High resale value:** You picked a make and model that holds its value well, giving you confidence that it’s a good long-term investment.  

#### Premortem (What-If) Decision Analysis 

Imagining successful outcomes helps you focus on actions like sticking to a car budget, getting a full inspection, and choosing a reliable model. You will be more likely to make a purchase you’ll be happy with. By plyaing a "game" of What-If with decision criteria in advance, you could adapt your strategy. For example, you could consider to thoroughly check maintenance records, buy additional warranty, test drive with similar terrain in your area, or reassess your budget to avoid these potential problems.

How can Generative AI help? Using Generative AI, the LLM models can be used to "imagine" premortem outcomes and  brainstorm potential factors why that certain outcome occured. Generative AI agents can act as your "premortem" team. This can lead to additional questions, gathering of intelligence, checklists, negotiation leverage etc. you didn't consider initially. 

### Step 1 - Initialize Configuration Builder & Build the Semantic Kernel Orchestration 

In [1]:
#r "nuget: Microsoft.Extensions.Configuration, 9.0.6"
#r "nuget: Microsoft.Extensions.Configuration.Json, 9.0.6"
#r "nuget: Microsoft.SemanticKernel, 1.56"
#r "nuget: Microsoft.SemanticKernel.Plugins.Core, 1.56-preview"

using Microsoft.Extensions.Configuration.Json;
using Microsoft.Extensions.Configuration;
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.ChatCompletion;
using Microsoft.SemanticKernel.Connectors.OpenAI;
using System.IO;

var configurationBuilder = new ConfigurationBuilder()
    .SetBasePath(Directory.GetCurrentDirectory())
    .AddJsonFile("local.settings.json", optional: true, reloadOnChange: true)
    .AddJsonFile("secrets.settings.json", optional: true, reloadOnChange: true);
var config = configurationBuilder.Build();

// IMPORTANT: You ONLY NEED either Azure OpenAI or OpenAI connectiopn info, not both.
// Azure OpenAI Connection Info using o1 series of models
var azureOpenAIEndpoint = config["AzureOpenAI:Endpoint"];
var azureOpenAIAPIKey = config["AzureOpenAI:APIKey"];
var azureOpenAIModelDeploymentName = config["AzureOpenAI:ModelDeploymentName"];
// OpenAI Connection Info 
var openAIAPIKey = config["OpenAI:APIKey"];
var openAIModelId = config["OpenAI:ModelId"];

Installed Packages Microsoft.Extensions.Configuration, 9.0.6 Microsoft.Extensions.Configuration.Json, 9.0.6 Microsoft.SemanticKernel, 1.56.0 Microsoft.SemanticKernel.Plugins.Core, 1.56.0-preview

In [2]:
using Microsoft.Extensions.DependencyInjection.Extensions;
using Microsoft.Extensions.DependencyInjection;
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.ChatCompletion;
using Microsoft.SemanticKernel.Connectors.OpenAI;

Kernel semanticKernel;

// Set the flag to use Azure OpenAI or OpenAI. False to use OpenAI, True to use Azure OpenAI
var useAzureOpenAI = true;

// Create a new Semantic Kernel instance
if (useAzureOpenAI)
{
    Console.WriteLine("Using Azure OpenAI Service");
    semanticKernel = Kernel.CreateBuilder()
        .AddAzureOpenAIChatCompletion(
            deploymentName: azureOpenAIModelDeploymentName,
            endpoint: azureOpenAIEndpoint,
            apiKey: azureOpenAIAPIKey)
        .Build();
}
else
{
    Console.WriteLine("Using OpenAI Service");
    semanticKernel = Kernel.CreateBuilder()
        .AddOpenAIChatCompletion(
            modelId: openAIModelId,
            apiKey: openAIAPIKey)
        .Build();
}

var chatCompletionService = semanticKernel.Services.GetRequiredService<IChatCompletionService>();

Using Azure OpenAI Service


### Step 2 - Premortem Risk Analysis with AI Reasoning Models

> 📜 **_"Success is a lousy teacher. It seduces smart people into thinking they can’t lose."_**
>
> -- <cite>Bill Gates (Co-Founder of Microsoft, Philantropist)</cite>  

In the decision scenario below, an AI model will perform a premortem (What-If) analysis on the decision whether to purchase or lease a car. The AI model will provide risks and items to consider for each of the decision options: purchasing a car or leasing a car. 

In [3]:
// Create a system prompt instruction to override the default system prompt
// Add the System Prompt (Persona) to behave like a decision intelligence assistant
var systemPromptForDecisions = """
You are a Decision Intelligence Assistant. 
Assist the user in exploring options, reasoning through decisions, problem-solving.
Apply systems thinking to various scenarios. Provide structured, logical, and comprehensive advice.
""";
var decisionPreMortemAnalysisPrompt = """
Perform an advanced premortem (What-If) analysis on the following decision scenario:
You are deciding to purchase a new car versus leasing a car.
""";

// Create a new chat history with pre-mortem analysis prompt
var chatHistoryPreMortemAnalysis = new ChatHistory();
chatHistoryPreMortemAnalysis.AddSystemMessage(systemPromptForDecisions);
chatHistoryPreMortemAnalysis.AddUserMessage(decisionPreMortemAnalysisPrompt);

// Create a new OpenAI prompt execution settings object
// Try different settings (Temperature, FrequencyPenalty etc) to see how they affect the quality of the generated text
var openAIPromptExecutionSettings = new OpenAIPromptExecutionSettings { 
    MaxTokens = 1000, 
    Temperature = 0.3, 
    TopP = 1.0, 
    FrequencyPenalty = 0.0, 
    PresencePenalty = 0.0
    };

var decisionResponseChainOfThought = string.Empty;
// Include both the chat history and the prompt execution settings in the call to GetStreamingChatMessageContentsAsync
await foreach (var content in chatCompletionService.GetStreamingChatMessageContentsAsync(chatHistoryPreMortemAnalysis, openAIPromptExecutionSettings))
{
    decisionResponseChainOfThought += content;
    Console.Write(content);
}

Certainly! A **premortem analysis** is a proactive decision tool where you imagine that your decision has failed, and then work backward to determine what could lead to that failure. This helps you identify risks, blind spots, and mitigation strategies before committing.

Let’s apply an advanced premortem (What-If) analysis to the scenario: **Purchase a new car vs. Lease a car.**

---

## 1. **Define the Decision**

**Decision:** Should you purchase a new car or lease a car?

**Objective:** Obtain reliable personal transportation while optimizing for cost, flexibility, and personal preferences.

---

## 2. **Imagine Failure Scenarios**

Suppose, one year from now, you regret your decision. What went wrong? Let’s explore both options:

### **A. If You Purchased the Car**

#### What-Ifs (Potential Failure Points):

1. **Financial Strain**
   - What if your financial situation changes (job loss, unexpected expenses) and you can’t afford the payments or maintenance?
2. **Depreciation Shock

The large reasoning model provided a comprehensive set of criteria, risks, mitigation strategy and approaches for the two decisions. This is a great option, but it might be more ideal to provide this in an easy to consume table (checklist) for the user that is easy to compare and contrast. This What-If analysis is easy to consume on a mobile device or even print out and annotate. 

In [6]:
// Create a system prompt instruction to override the default system prompt
// Add the System Prompt (Persona) to behave like a Decision Intelligence assistant
var systemPromptForDecisions = """
You are a Decision Intelligence Assistant. 
Assist the user in exploring options, reasoning through decisions, problem-solving.
Apply systems thinking to various scenarios. Provide structured, logical, and comprehensive advice.
""";
var decisionPreMortemAnalysisChecklistPrompt = """
Perform an advanced premortem (What-If) analysis on the following scenario:
You are deciding to purchase a new car versus leasing a car.

Output the risk factors, uncertainties, and potential outcomes ONLY in a Markdown table.
Create a column for each option (Purchase, Lease) and which risk factor or uncertainty applies to each option.
Do not surround the markdown with triple backticks.
""";

// Create a new chat history with pre-mortem analysis prompt
var chatHistoryPreMortemAnalysisChecklist = new ChatHistory();
chatHistoryPreMortemAnalysisChecklist.AddSystemMessage(systemPromptForDecisions);
chatHistoryPreMortemAnalysisChecklist.AddUserMessage(decisionPreMortemAnalysisChecklistPrompt);

// Create a new OpenAI prompt execution settings object
// Try different settings (Temperature, FrequencyPenalty etc) to see how they affect the quality of the generated text
var openAIPromptExecutionSettings = new OpenAIPromptExecutionSettings { 
    MaxTokens = 1000, 
    Temperature = 0.3, 
    TopP = 1.0, 
    FrequencyPenalty = 0.0, 
    PresencePenalty = 0.0
    };

var preMortemAnalysisResponse = string.Empty;
// Include both the chat history and the prompt execution settings in the call to GetStreamingChatMessageContentsAsync
await foreach (var content in chatCompletionService.GetStreamingChatMessageContentsAsync(chatHistoryPreMortemAnalysisChecklist, openAIPromptExecutionSettings))
{
    preMortemAnalysisResponse += content;
    Console.Write(content);
}

// Add the final response to the chat history for the pre-mortem analysis checklist history
chatHistoryPreMortemAnalysisChecklist.AddAssistantMessage(preMortemAnalysisResponse);

| **Risk Factor/Uncertainty**                  | **Purchase**                                                                 | **Lease**                                                                 |
|---------------------------------------------|-----------------------------------------------------------------------------|---------------------------------------------------------------------------|
| **Upfront Costs**                           | High initial cost due to down payment and taxes.                           | Lower upfront costs, typically just a smaller down payment or fees.       |
| **Monthly Payments**                        | Higher monthly payments if financing the car.                              | Lower monthly payments compared to purchasing.                            |
| **Depreciation**                            | Risk of rapid depreciation reducing resale value.                          | Not a concern, as the car is returned at the end of the lease.     

| **Risk Factor/Uncertainty**                  | **Purchase**                                                                 | **Lease**                                                                 |
|---------------------------------------------|-----------------------------------------------------------------------------|---------------------------------------------------------------------------|
| **Upfront Costs**                           | High initial cost due to down payment and taxes.                           | Lower upfront costs, typically just a smaller down payment or fees.       |
| **Monthly Payments**                        | Higher monthly payments if financing the car.                              | Lower monthly payments compared to purchasing.                            |
| **Depreciation**                            | Risk of rapid depreciation reducing resale value.                          | Not a concern, as the car is returned at the end of the lease.            |
| **Ownership**                               | Full ownership after payments are complete, with potential long-term value.| No ownership; must return the car at the end of the lease term.           |
| **Mileage Restrictions**                    | No restrictions; freedom to drive as much as needed.                       | Mileage limits may incur costly penalties if exceeded.                    |
| **Maintenance Costs**                       | Responsible for all maintenance and repair costs after warranty expires.   | Typically covered under warranty during the lease term.                   |
| **Flexibility**                             | Less flexibility; harder to switch vehicles without selling or trading in. | Easier to switch to a new car at the end of the lease term.               |
| **Customization**                           | Can modify or customize the car as desired.                                | Limited or no ability to customize the car.                               |
| **Long-Term Financial Impact**              | Potential for better financial value if the car is kept long-term.         | May cost more over time if leasing repeatedly without purchasing.         |
| **Residual Value Risk**                     | Risk of lower resale value than expected.                                  | No residual value risk; the leasing company assumes this risk.            |
| **Technology Obsolescence**                 | Risk of owning a car that becomes outdated quickly.                        | Easier to upgrade to newer models with the latest technology.             |
| **Contractual Obligations**                 | No contractual restrictions after purchase.                                | Bound by lease terms with penalties for early termination.                |
| **Insurance Costs**                         | May have slightly lower insurance requirements.                            | May require higher insurance coverage as per lease agreement.             |
| **Tax Implications**                        | May qualify for tax deductions (e.g., business use).                       | Lease payments may be partially deductible for business use.              |
| **End-of-Term Options**                     | Can sell, trade-in, or keep the car indefinitely.                          | Must return the car or purchase it at residual value.                     |
| **Market Conditions**                       | Resale value depends on used car market conditions.                        | Lease terms are fixed, unaffected by market fluctuations.                 |
| **Emotional Attachment**                    | Higher likelihood of attachment to the car due to ownership.               | Less emotional attachment as the car is temporary.                        |

This Markdown table can be further simplified by highlighting which risk factor is more favorable when comparing a car purchase versus a car lease. This can make the decision process much easier to the user, as at a glance they can see overwhelming evidence that favors once decision option over the other.  

In [7]:
var simplifyRiskAnalysisPrompt = """
You are provided a detailed markdown table in the above chat history that 
compares Purchase and Lease options across various risk factors and uncertainties. 
Your task is to convert this table into a simplified markdown table that uses check marks (☑️) 
to indicate which option is less risky or more ideal for each risk factor. 
The output table should have three columns: "Risk Factor / Uncertainty / Potential Outcome", "Purchase", and "Lease". 
For each risk factor, decide based on the detailed descriptions which option (Purchase or Lease) 
is preferable (i.e., less risky or more advantageous) and place a check mark (☑️) in that column. 
If an option is not the ideal choice for a given risk factor, leave its cell blank. 
Make sure the simplified table closely reflects the key insights from the detailed table. 
""";

chatHistoryPreMortemAnalysisChecklist.AddUserMessage(simplifyRiskAnalysisPrompt);

var preMortemAnalysisSimplifiedChecklistResponse = string.Empty;
// Include both the chat history and the prompt execution settings in the call to GetStreamingChatMessageContentsAsync
await foreach (var content in chatCompletionService.GetStreamingChatMessageContentsAsync(chatHistoryPreMortemAnalysisChecklist, openAIPromptExecutionSettings))
{
    preMortemAnalysisSimplifiedChecklistResponse += content;
    Console.Write(content);
}

| **Risk Factor / Uncertainty / Potential Outcome** | **Purchase** | **Lease** |
|---------------------------------------------------|--------------|-----------|
| **Upfront Costs**                                 |              | ☑️        |
| **Monthly Payments**                              |              | ☑️        |
| **Depreciation**                                  |              | ☑️        |
| **Ownership**                                     | ☑️           |           |
| **Mileage Restrictions**                          | ☑️           |           |
| **Maintenance Costs**                             |              | ☑️        |
| **Flexibility**                                   |              | ☑️        |
| **Customization**                                 | ☑️           |           |
| **Long-Term Financial Impact**                    | ☑️           |           |
| **Residual Value Risk**                           |              | ☑️        |
| **Technology Obsolescence*

| **Risk Factor / Uncertainty / Potential Outcome** | **Purchase** | **Lease** |
|---------------------------------------------------|--------------|-----------|
| **Upfront Costs**                                 |              | ☑️        |
| **Monthly Payments**                              |              | ☑️        |
| **Depreciation**                                  |              | ☑️        |
| **Ownership**                                     | ☑️           |           |
| **Mileage Restrictions**                          | ☑️           |           |
| **Maintenance Costs**                             |              | ☑️        |
| **Flexibility**                                   |              | ☑️        |
| **Customization**                                 | ☑️           |           |
| **Long-Term Financial Impact**                    | ☑️           |           |
| **Residual Value Risk**                           |              | ☑️        |
| **Technology Obsolescence**                       |              | ☑️        |
| **Contractual Obligations**                       | ☑️           |           |
| **Insurance Costs**                               | ☑️           |           |
| **Tax Implications**                              | ☑️           | ☑️        |
| **End-of-Term Options**                           | ☑️           |           |
| **Market Conditions**                             |              | ☑️        |
| **Emotional Attachment**                          | ☑️           |           |

### Step 3 - Premortem Analysis for Radio Telescope Operations 

Exploring space with telescopes is a time consuming investment. Current telescope designs are limited to exploring small areas of space in a certain time frame. Without going into details, the process is much more complex than snapping a picture with a mobile device. Telescopes can be pointed at a specific space area for days, weeks or even more than months! If a telescope is exploring a certain area of space it isn't collecting data on other areas. Scientists and researchers compete for valuable access to telescope resources by providing observatories what space area they want to explore, any cellestial contraints, timeframes, calibration costs, data collection transfer etc. This presents a very interesting Decision Intelligence constraints problem that observatories must manage. 

<img style="display: block; margin: auto;" width ="700px" src="https://raw.githubusercontent.com/bartczernicki/DecisionIntelligence.GenAI.Workshop/main/Images/Scenarios/Scenario-RadioTelescope.png">

#### Decision Scenario - Premortem for Radio Telescope Operations 

> 📜 "We choose to go to the Moon in this decade and do the other things, not because they are easy, but because they are hard." 
>
> -- <cite>John F. Kennedy (35th President of the United States)</cite>  

A newly commissioned radio telescope stands on the verge of unveiling the proof of alien life, but it is still far from fully calibrated. Despite limited data processing capacity and a software suite that remains untested, an unexpected cosmic event has just begun, hinting at signals that could reveal alien life. Faced with the risk of hardware damage or data corruption from running the telescope prematurely, scientists debate whether to switch it on now or wait until everything is fully operational—knowing they could lose a once-in-a-generation opportunity. Determined not to squander this brief cosmic window, the team conducts a premortem analysis to predict what might go wrong. Armed with these insights, they can better prepare for potential pitfalls and strengthen their chances of catching historic signals from across the galaxy.

Let's assume the scientists decide to rush the radio telescope to operate early to find alien life. Furthermore, assume a that this decision has led to a disaster with hardware failure to the telescope. Even though the (What-If) decision was made to rush the radio telescope operationally, formal scientific telescope operations have been followed. The goal of this premortem (What-If analysis) is to illuminate any risk mitigation strategies and craft a risk mitigation approach that may have been uncovered based on the premortem. 

Techinques like this a very powerful in a variety of business scenarios, where existing operational frameworks exist but simulating an outcome can uncover gaps, emerging risks and other opportunities to optimize. 

In [9]:
// Create a system prompt instruction to override the default system prompt
// Add the System Prompt (Persona) to behave like a decision intelligence assistant
var systemPromptForPremortemRadioTelescope = """
You are a Decision Intelligence Assistant. 
Assist the user in exploring options, reasoning through decisions, problem-solving.
Apply systems thinking to various scenarios. Provide structured, logical, and comprehensive advice.
""";
var decisionAnalysisPromptForPremortemRadioTelescope = """
You are a project manager at a new radio telescope facility looking for alien life.  
It is still far from fully calibrated. Despite limited data processing capacity and a software suite that remains untested, 
an unexpected cosmic event has just begun, hinting at signals that could reveal alien life. 
Faced with the risk of hardware damage or data corruption from running the telescope prematurely, 
scientists debate whether to switch it on now or wait until everything is fully operational—knowing 
they could lose a once-in-a-generation opportunity. 

Imagine it is six months from now and the project to detect potential alien signals from the sudden cosmic event has failed. 
The radio telescope was likely activated prematurely, software remained untested, and the team rushed to capture transmissions. 
Despite the urgency, the outcome was disastrous: hardware suffered damage under the increased load, 
data came out corrupted, or no meaningful signals were confirmed. 

Now, step into that moment of failure—what specific issues, oversights, or cascading errors caused this grim scenario?

1) Envision every possible way the radio telescope's early deployment could have led to compromised or unusable data.
2) Identify how incomplete software testing and rushed calibration might have 
contributed to errors in processing or storing signals. 
3) Consider any overlooked safety protocols or hardware precautions that might have led to damage or malfunctions.
4) Examine the roles and responsibilities of the team and think about communication breakdowns, 
rushed decisions, and missed signals or deadlines.
5) Assess how time pressure from the rare cosmic event might have amplified smaller mistakes and 
turned them into systemic failures.

Use the outcomes of this “what if” vision to create a detailed list of preventive measures. 
Then, propose fail-safes, additional resources, or procedural changes to ensure that, 
should you choose to activate the telescope early, you can still capture the event safely and 
effectively—maximizing the chance to detect historic signals without jeopardizing the entire operation.
""";

// Create a new chat history with pre-mortem analysis prompt
var chatHistoryPreMortemTelescopeAnalysis = new ChatHistory();
chatHistoryPreMortemTelescopeAnalysis.AddSystemMessage(systemPromptForPremortemRadioTelescope);
chatHistoryPreMortemTelescopeAnalysis.AddUserMessage(decisionAnalysisPromptForPremortemRadioTelescope);

// Create a new OpenAI prompt execution settings object
// Try different settings (Temperature, FrequencyPenalty etc) to see how they affect the quality of the generated text
var openAIPromptExecutionSettings = new OpenAIPromptExecutionSettings { 
    MaxTokens = 1000, 
    Temperature = 0.3, 
    TopP = 1.0, 
    FrequencyPenalty = 0.0, 
    PresencePenalty = 0.0
    };

var preMortemTelescopeAnalysisResponse = string.Empty;
// Include both the chat history and the prompt execution settings in the call to GetStreamingChatMessageContentsAsync
await foreach (var content in chatCompletionService.GetStreamingChatMessageContentsAsync(chatHistoryPreMortemTelescopeAnalysis, openAIPromptExecutionSettings))
{
    preMortemTelescopeAnalysisResponse += content;
    Console.Write(content);
}

### Post-Failure Analysis: Identifying Causes of the Grim Scenario

#### 1. **Compromised or Unusable Data**
   - **Signal Interference:** The telescope's uncalibrated state led to misaligned antennas, causing signal distortion or overlap.
   - **Data Corruption:** The untested software suite failed to handle the high data throughput, resulting in corrupted files or incomplete datasets.
   - **Noise Misclassification:** Without proper calibration, the system misclassified cosmic noise or Earth-origin interference as potential signals, rendering the data unreliable.
   - **Insufficient Storage Management:** The team underestimated the storage requirements, leading to overwriting or loss of critical data.
   - **Inadequate Signal Filtering:** Algorithms designed to isolate alien signals were either incomplete or malfunctioning, leading to false positives or missed detections.

#### 2. **Errors from Incomplete Software Testing and Rushed Calibration**
   - **Software Bugs:** Unvetted code